In [1]:
import os
from geomltoolkits.downloader import oam as OAMDownloader
from geomltoolkits.downloader import osm as OSMDownloader

## Define area of interest to work on 

- Go to Open Aerial Map and then find TMS (Copy TMS Link)
    For this example we are using this image : https://map.openaerialmap.org/#/85.52534580230713,27.63552012302292,14/square/12313122310120202/62d86c65d8499800053796c4?_k=okhkbl 

- Verify OSM is mapped properly in that area
    Go to www.osm.org  , You can simply switch layer in OAM itself and see osm underneath 
    
- Define area of interest where you want to generate training data 
    You can get your bbox from your geojson / area from here : https://kshitijrajsharma.github.io/geojson2bbox/ 

In [6]:
# Default parameters
ZOOM = 18
WORK_DIR = "banepa"
TMS = "https://tiles.openaerialmap.org/62d85d11d8499800053796c1/0/62d85d11d8499800053796c2/{z}/{x}/{y}"
# AOI = "input.geojson"
bbox = [85.51678033745037,27.6313353660439,85.52323021107895,27.637438390948745]

# Ensure working directory
os.makedirs(WORK_DIR, exist_ok=True)

### Generate chips images for your area of interest

In [7]:
# Download tiles
await OAMDownloader.download_tiles(
        tms=TMS, 
        zoom=ZOOM, 
        out=WORK_DIR, 
        bbox=bbox, 
        georeference=True, 
        dump=True,
        prefix="OAM"
    )


Total tiles fetched: 36


  0%|          | 0/36 [00:00<?, ?tile/s]

  3%|▎         | 1/36 [00:01<00:45,  1.29s/tile]/Users/krschap/hotosm/geoml-toolkits/env/lib/python3.12/site-packages/rasterio/__init__.py:358: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = get_writer_for_path(path, driver=driver)(
100%|██████████| 36/36 [00:05<00:00,  6.99tile/s]


### Fetch labels 

Here we are using the same tiles.geojson that was used to fetch tiles , remember your bbox / geojson and tile boundary might be little bit different due to tile size and we want to make sure we have labels for the tile boundary hence input here would be tiles.geojson 

In [9]:
# Download OSM data for tile boundary
tiles_geojson = os.path.join(WORK_DIR, "tiles.geojson")
await OSMDownloader.download_osm_data(
    geojson=tiles_geojson, 
    out=os.path.join(WORK_DIR, "labels"),
    dump=True
)


OSM Data Last Updated :  2025-01-23 12:07:05+00:00


{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[85.5188026, 27.6314281],
      [85.5188842, 27.6314107],
      [85.5188998, 27.6314684],
      [85.5189006, 27.6314713],
      [85.518819, 27.6314887],
      [85.5188182, 27.6314858],
      [85.5188026, 27.6314281]]]},
   'properties': {'osm_id': 345211057,
    'osm_type': 'ways_poly',
    'version': 3,
    'tags': {'building': 'yes'},
    'changeset': 126803182,
    'timestamp': '2022-09-30T03:24:29'}},
  {'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[85.5188973, 27.6314483],
      [85.5189699, 27.6314323],
      [85.5189831, 27.6314793],
      [85.5189106, 27.6314953],
      [85.5188973, 27.6314483]]]},
   'properties': {'osm_id': 345211020,
    'osm_type': 'ways_poly',
    'version': 5,
    'tags': {'building': 'yes',
     'roof:shape': 'mixed',
     'roof:material': 'concrete'},
    'changeset': 126803182,
    'timestamp': '2022-09-3